# Semantic Kernel

ဤကုဒ်နမူနာတွင် [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI Framework ကို အသုံးပြု၍ အခြေခံ Agent တစ်ခုကို ဖန်တီးမည်ဖြစ်သည်။

ဤနမူနာ၏ ရည်ရွယ်ချက်မှာ နောက်ထပ်ကုဒ်နမူနာများတွင် အမျိုးမျိုးသော Agentic ပုံစံများကို အကောင်အထည်ဖော်ရာတွင် အသုံးပြုမည့် လုပ်ငန်းစဉ်များကို ပြသရန် ဖြစ်သည်။


In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## Client ဖန်တီးခြင်း

ဤနမူနာတွင် [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ကို အသုံးပြု၍ LLM ကို အသုံးပြုမည်ဖြစ်သည်။

`ai_model_id` ကို `gpt-4o-mini` အဖြစ် သတ်မှတ်ထားသည်။ GitHub Models marketplace တွင် ရရှိနိုင်သည့် အခြားမော်ဒယ်တစ်ခုကို ပြောင်းလဲကြည့်ပြီး မတူညီသော ရလဒ်များကို ကြည့်ရှုနိုင်ပါသည်။

GitHub Models အတွက် `base_url` ကို အသုံးပြုသည့် `Azure Inference SDK` ကို အသုံးပြုရန်အတွက်၊ Semantic Kernel အတွင်းရှိ `OpenAIChatCompletion` connector ကို အသုံးပြုမည်ဖြစ်သည်။ အခြားသော မော်ဒယ်ပံ့ပိုးသူများအတွက် Semantic Kernel ကို အသုံးပြုနိုင်ရန် [ရရှိနိုင်သော အခြား connector များ](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) လည်း ရှိပါသည်။


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Agent ကိုဖန်တီးခြင်း

အောက်တွင် `TravelAgent` ဟုခေါ်သော Agent ကိုဖန်တီးထားပါသည်။

ဤဥပမာအတွက် ကျွန်ုပ်တို့သည် အလွန်ရိုးရှင်းသောညွှန်ကြားချက်များကို အသုံးပြုထားပါသည်။ သင်သည် ဤညွှန်ကြားချက်များကို ပြောင်းလဲပြီး Agent ရဲ့ တုံ့ပြန်မှုကွဲပြားမှုကို ကြည့်ရှုနိုင်ပါသည်။


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## အေးဂျင့်များကို အလုပ်လုပ်စေခြင်း

အခုတော့ `ChatHistory` ကို သတ်မှတ်ပြီး `system_message` ကို ထည့်သွင်းခြင်းဖြင့် အေးဂျင့်ကို အလုပ်လုပ်စေနိုင်ပါပြီ။ အရင်က သတ်မှတ်ထားသော `AGENT_INSTRUCTIONS` ကို အသုံးပြုမည်ဖြစ်သည်။

ဤအချက်များကို သတ်မှတ်ပြီးနောက်၊ အသုံးပြုသူက အေးဂျင့်ထံ ပေးပို့မည့် `user_inputs` ကို ဖန်တီးမည်ဖြစ်သည်။ ဤကိစ္စတွင် `Plan me a sunny vacation` ဟု သတ်မှတ်ထားသည်။

အေးဂျင့်၏ တုံ့ပြန်မှုကို မတူကွဲပြားစေရန်၊ သင်၏ စာတိုကို ပြောင်းလဲပေးနိုင်ပါသည်။


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

**ဝက်ဘ်ဆိုက်မှတ်ချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ဆိုမှုများတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို ကျေးဇူးပြု၍ သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူလဘာသာစကားဖြင့် အာဏာတည်သောရင်းမြစ်အဖြစ် သတ်မှတ်ရန် လိုအပ်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ်ဘာသာပြန်ဆိုမှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်ဆိုမှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားများ သို့မဟုတ် အဓိပ္ပါယ်မှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
